In [1]:
import re

import numpy as np
import pandas as pd

In [2]:
from IPython.display import HTML, display

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)


def pp(df):
    if isinstance(df, pd.Series):
        df = df.to_frame()
    return display(HTML(df.to_html().replace("\\n", "<br>")))

In [3]:
noprompt = pd.read_parquet("noprompt.parquet")
original = pd.read_parquet("original.parquet")
source = pd.read_parquet("source.parquet")

In [4]:
df = pd.merge(noprompt, original, on="hash", suffixes=("_np", "_og"))
df = pd.merge(df, source, on="hash", suffixes=("", "_src"))
df = df.drop(columns=[c for c in df.columns if "gold" in c and "_" in c])


def clean_quotes(x):
    return re.sub('\\"+', '"', x)


for part in ["causes", "effects"]:
    for kind in ["np", "og"]:
        df[f"pred.{part}_{kind}"] = df[f"pred.{part}_{kind}"].apply(clean_quotes)
    df[f"gold,{part}"] = df[f"gold.{part}"].apply(clean_quotes)
df["context"] = df["context"].apply(clean_quotes)

df.columns

Index(['pred.causes_np', 'pred.effects_np', 'pred.relation_np', 'correct_np',
       'hash', 'pred.causes_og', 'pred.effects_og', 'pred.relation_og',
       'correct_og', 'gold.causes', 'gold.effects', 'gold.relation', 'context',
       'gold,causes', 'gold,effects'],
      dtype='object')

In [5]:
dd = df.query("~correct_np and ~correct_og")

for i in range(10):
    pp(dd.iloc[i])

,0
pred.causes_np,"bb&t and suntrust have completed their merger, forming truist"
pred.effects_np,drive the next step up in profitability for the franchises
pred.relation_np,enable
correct_np,False
hash,79460147
pred.causes_og,"bb&t and suntrust have completed their merger, forming truist"
pred.effects_og,drive the next step up in profitability for the franchises
pred.relation_og,enable
correct_og,False
gold.causes,"bb&t and suntrust have completed their merger, forming truist"


,1
pred.causes_np,tulip's lack of profitability
pred.effects_np,we do not believe the business maintains a cost advantage
pred.relation_np,cause
correct_np,False
hash,ec89c529
pred.causes_og,tulip's lack of profitability
pred.effects_og,we do not believe the business maintains a cost advantage
pred.relation_og,cause
correct_og,False
gold.causes,given tulip's lack of profitability (management has stated the business was not profitable at the time of the october 2019 acquisition)


,5
pred.causes_np,management will continue to pursue acquisition targets with high levels of recurring revenue
pred.effects_np,further lower the cyclicality of fortive's portfolio
pred.relation_np,cause
correct_np,False
hash,a38bd335
pred.causes_og,management will continue to pursue acquisition targets with high levels of recurring revenue
pred.effects_og,further lower the cyclicality of fortive's portfolio
pred.relation_og,cause
correct_og,False
gold.causes,management will continue to pursue acquisition targets with high levels of recurring revenue


,6
pred.causes_np,"beyond's products emit 90% less greenhouse gases, require 93% less land, 99% less water, and 46% less energy to produce than their meat equivalents"
pred.effects_np,a primary growth driver to be the 20% of consumers willing to adjust their habits to benefit the environment
pred.relation_np,cause
correct_np,False
hash,5e6ed070
pred.causes_og,"beyond's products emit 90% less greenhouse gases, require 93% less land, 99% less water, and 46% less energy to produce than their meat equivalents"
pred.effects_og,a primary growth driver to be the 20% of consumers willing to adjust their habits to benefit the environment
pred.relation_og,cause
correct_og,False
gold.causes,"beyond's products emit 90% less greenhouse gases, require 93% less land, 99% less water, and 46% less energy to produce"


,8
pred.causes_np,greater capital efficiency and business mix tilted toward credit cards
pred.effects_np,drive the bank's exceptional net interest margins
pred.relation_np,cause
correct_np,False
hash,6f2add46
pred.causes_og,greater capital efficiency and business mix tilted toward credit cards
pred.effects_og,"high-yield loans drive the bank's exceptional net interest margins, which routinely exceed those of money center banks by 600 basis points"
pred.relation_og,cause
correct_og,False
gold.causes,this is largely attributable to greater capital efficiency and business mix tilted toward credit cards


,11
pred.causes_np,gap lacks a competitive edge
pred.effects_np,its operating margins and returns on invested capital have declined since 2013
pred.relation_np,cause
correct_np,False
hash,a01f30f9
pred.causes_og,evidence that gap lacks a competitive edge
pred.effects_og,its operating margins and returns on invested capital have declined since 2013
pred.relation_og,cause
correct_og,False
gold.causes,evidence that gap lacks a competitive edge


,12
pred.causes_np,people start opting to use debit cards rather than credit cards
pred.effects_np,eat into discover's ability to generate interest income
pred.relation_np,cause
correct_np,False
hash,48cb1305
pred.causes_og,people start opting to use debit cards rather than credit cards
pred.effects_og,this could eat into discover's ability to generate interest income
pred.relation_og,cause
correct_og,False
gold.causes,people start opting to use debit cards


,13
pred.causes_np,licenses for these facilities are given by state and local officials
pred.effects_np,"if ethylene oxide becomes a priority for state officials, teleflex could face supply disruptions"
pred.relation_np,cause
correct_np,False
hash,e301836a
pred.causes_og,licenses for these facilities are given by state and local officials
pred.effects_og,"ethylene oxide becomes a priority for state officials, teleflex could face supply disruptions"
pred.relation_og,cause
correct_og,False
gold.causes,ethylene oxide becomes a priority for state officials | licenses for these facilities are given by state and local officials


,15
pred.causes_np,"we still view polaris as more seasonally and end-user diversified (it sells electric vehicles, military products, motorcycles, and more)"
pred.effects_np,brp's normalized operating margins could trail slightly behind
pred.relation_np,cause
correct_np,False
hash,5430f352
pred.causes_og,polaris as more seasonally and end-user diversified
pred.effects_og,we think brp's normalized operating margins could trail slightly behind
pred.relation_og,cause
correct_og,False
gold.causes,we still view polaris as more seasonally and end-user diversified


,19
pred.causes_np,the success of customer-friendly sales practices and use of information technology and the firm targets a 12% annual rate for fiscal 2021 to fiscal 2026
pred.effects_np,carmax's revenue has increased at a compound annual rate of about 11% since fiscal 2000
pred.relation_np,cause
correct_np,False
hash,53db011d
pred.causes_og,the success of customer-friendly sales practices and use of information technology and the firm targets a 12% annual rate for fiscal 2021 to fiscal 2026
pred.effects_og,carmax's revenue has increased at a compound annual rate of about 11% since fiscal 2000
pred.relation_og,cause
correct_og,False
gold.causes,the success of customer-friendly sales practices and use of information technology


In [6]:
cc = df.copy()
SEGMENTS = [(part, kind) for part in ["causes", "effects"] for kind in ["np", "og"]]


def cont(x, y):
    xs = [i.lower().strip() for i in x.split("|") if i.strip()]
    ys = [i.lower().strip() for i in y.split("|") if i.strip()]

    for x in xs:
        for y in ys:
            if x in y or y in x:
                return True

    return False


def _cont(part, kind):
    def f(row):
        gold = f"gold.{part}"
        pred = f"pred.{part}_{kind}"
        return cont(row[gold], row[pred])

    return f


for part, kind in SEGMENTS:
    cc[f"{part}_{kind}_cont"] = cc.apply(_cont(part, kind), axis="columns")


cont_s = pd.Series([True] * cc.shape[0])
for part, kind in SEGMENTS:
    cont_s &= cc[f"{part}_{kind}_cont"]

cc["cont"] = cont_s
cc["cont"].value_counts()

True     1447
False     448
Name: cont, dtype: int64

In [7]:
for i in range(10):
    pp(cc[~cc["cont"]].iloc[i])

,4
pred.causes_np,recently generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat covid-19
pred.effects_np,we are keeping our fair value estimate for danaher at $160 per share after raising it in january 2021
pred.relation_np,cause
correct_np,True
hash,5a3ca6d1
pred.causes_og,recent generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat covid-19
pred.effects_og,we are keeping our fair value estimate for danaher at $160 per share after raising it in january 2021
pred.relation_og,cause
correct_og,False
gold.causes,recently generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat covid-19


,45
pred.causes_np,sizable r&d expenses
pred.effects_np,critical to the firm's ability to sustain its advantage
pred.relation_np,enable
correct_np,False
hash,25dd5287
pred.causes_og,intel's historical lead in process technology benefits from sizable r&d expenses
pred.effects_og,critical to the firm's ability to sustain its advantage
pred.relation_og,enable
correct_og,False
gold.causes,sizable r&d expenses


,50
pred.causes_np,the concentration of high-quality tenants across simon's portfolio
pred.effects_np,provide an experience for consumers that is hard for other retailers to replicate
pred.relation_np,enable
correct_np,False
hash,86564276
pred.causes_og,the coronavirus
pred.effects_og,significant uncertainty about the future of sales growth
pred.relation_og,cause
correct_og,False
gold.causes,the concentration of high-quality tenants across simon's portfolio


,56
pred.causes_np,defective equipment or components from abb's automation or electrical portfolio
pred.effects_np,product recalls the product portfolio is diverse [causes] we would not expect any specific recalls to be material across the group financials
pred.relation_np,cause
correct_np,False
hash,e3aad36b
pred.causes_og,"the product portfolio is diverse, and we believe the company has internal controls to minimize recall risk"
pred.effects_og,defective equipment or components from abb's automation or electrical portfolio could lead to product recalls
pred.relation_og,cause
correct_og,False
gold.causes,defective equipment or components from abb's automation or electrical portfolio


,61
pred.causes_np,a lack of pricing power persists with petrol product pricing determined by the government
pred.effects_np,sinopec does benefit at home from its leadership position in downstream operations
pred.relation_np,cause
correct_np,False
hash,1ed43683
pred.causes_og,a lack of pricing power persists with petrol product pricing determined by the government
pred.effects_og,sinopec does benefit at home from its leadership position in downstream operations
pred.relation_og,cause
correct_og,False
gold.causes,petrol product pricing determined by the government


,65
pred.causes_np,payroll providers are handling increasing amounts of sensitive data
pred.effects_np,could be compromised in a cyberattack
pred.relation_np,cause
correct_np,False
hash,0d4c3918
pred.causes_og,payroll providers are handling increasing amounts of sensitive data
pred.effects_og,be compromised in a cyberattack
pred.relation_og,cause
correct_og,False
gold.causes,payroll providers are handling increasing amounts of sensitive data that could be compromised in a cyberattack


,75
pred.causes_np,the immense financial losses an upstart would incur while trying to amass the volume and density necessary to absorb the remarkably high capital outlays and fixed costs associated with a global parcel delivery network
pred.effects_np,this
pred.relation_np,cause
correct_np,False
hash,920ac7de
pred.causes_og,the immense financial losses an upstart would incur while trying to amass the volume and density necessary to absorb the remarkably high capital outlays and fixed costs associated with a global parcel delivery network
pred.effects_og,this
pred.relation_og,cause
correct_og,False
gold.causes,amass the volume and density necessary to absorb the remarkably high capital outlays and fixed costs associated with a global parcel delivery network


,77
pred.causes_np,large global cros like ppd have developed regulatory expertise and proprietary data
pred.effects_np,"help them produce the best trial designs and enroll patients into trials quickly and efficiently, allowing drug manufacturers - specialty & genericdrugs to get to market faster"
pred.relation_np,enable
correct_np,False
hash,d5e145d6
pred.causes_og,large global cros like ppd have developed regulatory expertise and proprietary data
pred.effects_og,drug manufacturers - specialty & genericdrugs to get to market faster
pred.relation_og,enable
correct_og,False
gold.causes,developed regulatory expertise and proprietary data


,78
pred.causes_np,north american gas is not some of the cheapest gas on the global cost curve
pred.effects_np,"without the contracts in place, cheniere would struggle"
pred.relation_np,cause
correct_np,False
hash,7de27477
pred.causes_og,north american gas is not some of the cheapest gas on the global cost curve
pred.effects_og,"without the contracts in place, cheniere would struggle"
pred.relation_og,cause
correct_og,False
gold.causes,without the contracts in place


,86
pred.causes_np,significant barriers to entry and high switching costs
pred.effects_np,market share has remained stable in the orthopedic implant space
pred.relation_np,cause
correct_np,False
hash,e17f0345
pred.causes_og,significant barriers to entry and high switching costs
pred.effects_og,market share has remained stable in the orthopedic implant space
pred.relation_og,cause
correct_og,False
gold.causes,"significant barriers to entry and high switching costs, market share has remained stable in the orthopedic implant space"


In [8]:
def count_clauses(x):
    return len(x.split("|"))


for part in ["causes", "effects"]:
    for kind in ["np", "og"]:
        cc[f"n_pred_{part}_{kind}"] = cc[f"pred.{part}_{kind}"].apply(count_clauses)
    cc[f"n_gold_{part}"] = cc[f"gold.{part}"].apply(count_clauses)

cc.columns

Index(['pred.causes_np', 'pred.effects_np', 'pred.relation_np', 'correct_np',
       'hash', 'pred.causes_og', 'pred.effects_og', 'pred.relation_og',
       'correct_og', 'gold.causes', 'gold.effects', 'gold.relation', 'context',
       'gold,causes', 'gold,effects', 'causes_np_cont', 'causes_og_cont',
       'effects_np_cont', 'effects_og_cont', 'cont', 'n_pred_causes_np',
       'n_pred_causes_og', 'n_gold_causes', 'n_pred_effects_np',
       'n_pred_effects_og', 'n_gold_effects'],
      dtype='object')

In [9]:
n_clauses = pd.Series([0] * cc.shape[0])

for part in ["causes", "effects"]:
    for kind in ["np", "og"]:
        # n_clauses &=  cc[f'n_pred_{part}_{kind}'] == 1
        n_clauses = np.maximum(n_clauses, cc[f"n_pred_{part}_{kind}"])
    n_clauses = np.maximum(n_clauses, cc[f"n_gold_{part}"])
    # n_clauses &= cc[f'n_gold_{part}']

n_clauses.value_counts()

1    1867
2      24
3       4
dtype: int64

In [10]:
single = cc[n_clauses == 1].copy()
single.shape

(1867, 26)

In [11]:
ss = single.copy()


def cont_ss(part, kind):
    def f(row):
        pred_col = f"pred.{part}_{kind}"
        pred = row[pred_col].lower().strip()
        ctx = row["context"].lower().strip()
        return pred in ctx

    return f


for part, kind in SEGMENTS:
    ss[f"{part}_{kind}_cont_ss"] = ss.apply(cont_ss(part, kind), axis="columns")


cont_ss = pd.Series([True] * ss.shape[0])
for part, kind in SEGMENTS:
    cont_ss &= ss[f"{part}_{kind}_cont_ss"]

ss["cont_ss"] = cont_ss
ss["cont_ss"].fillna(False, inplace=True)
ss["cont_ss"].value_counts(dropna=False)

True     1741
False     126
Name: cont_ss, dtype: int64

In [12]:
ss.groupby(["cont_ss", "correct_np", "correct_og"]).size()

cont_ss  correct_np  correct_og
False    False       False          100
                     True            18
         True        False            3
                     True             5
True     False       False         1136
                     True            81
         True        False           74
                     True           450
dtype: int64

In [13]:
kind = "og"

ss[f"{kind}_cont_ss"] = ss[f"causes_{kind}_cont_ss"] & ss[f"effects_{kind}_cont_ss"]
tmp = ss.loc[
    (~ss[f"{kind}_cont_ss"]),
    [
        "context",
        f"pred.causes_{kind}",
        f"pred.effects_{kind}",
        "correct_og",
        # "cont_ss",
    ]
    + [x for x in ss.columns if "cont_ss" in x and kind in x],
]
tmp.head()

,context,pred.causes_og,pred.effects_og,correct_og,causes_og_cont_ss,effects_og_cont_ss,og_cont_ss
4,We are keeping our fair value estimate for Danaher at $160 per share after raising it in January 2021 to reflect recently generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat COVID-19.,recent generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat covid-19,we are keeping our fair value estimate for danaher at $160 per share after raising it in january 2021,False,False,True,False
47,"Whereas the relative complexity involved in developing a facility had served as a deterrent, an increased focus on supply chains due to the rise of e-commerce and the coronavirus outbreak has eroded this prior source of hesitation.",the rise of e-commerce and the coronavirus outbreak,an increased focus on supply chains has eroded this prior source of hesitation,False,True,False,False
465,"As demand resumes longer term, we think pricing could grow at a low-single-digit clip (averaging around 3% between 2023-29) as willingness to socially gather resumes.",willingness to socially gather resumes,pricing could grow at a low-single-digit clip (averaging around 3% between 2023-239),False,True,False,False
489,"Second, it is possible that as new TLDs such as .flights gain popularity, demand for .com and .net declines, impacting the lucrative nature of Verisign's exclusive registry service for the two TLDs.","new tlds such as.flights gain popularity, demand for.com and.net declines",impacting the lucrative nature of verisign's exclusive registry service for the two tlds,False,False,True,False
811,"Finally, the end customer could risk production disruption or delays accompanying a modified product design. As a result, Sensata's sensors are likely to remain designed in for a product's entire lifecycle鈥攔anging from five to seven years for cars, all the way up to three decades for a commercial airplane.",the end customer could risk production disruption or delays accompanying a modified product design,"sensata's sensors are likely to remain designed in for a product's entire lifecycleanging from five to seven years for cars, all the way up to three decades for a commercial airplane",False,True,False,False


In [14]:
ss[f"{kind}_cont_ss"].value_counts()

True     1857
False      10
Name: og_cont_ss, dtype: int64

In [17]:
def check_tag(df, col):
    tag_re = "\[(?:causes|enables|prevents)\]"
    return df[col].str.contains(tag_re, regex=True)


tmp2 = tmp[
    check_tag(tmp, f"pred.causes_{kind}") | check_tag(tmp, f"pred.effects_{kind}")
]
max_i = min(10, tmp2.shape[0])
print(tmp2.shape)
for i in range(max_i):
    pp(tmp2.iloc[i])

(0, 7)


In [ ]:
ss["wrong_tag"] = check_tag(ss, "pred.causes_np") | check_tag(ss, "pred.effects_np")
ss["wrong_tag"].value_counts()

In [ ]:
ss.groupby([f"{kind}_cont_ss", "wrong_tag"], dropna=False).size()

In [25]:
xtmp = ss.loc[~ss[f"{kind}_cont_ss"]]
for i in range(xtmp.shape[0]):
    print(i)
    pp(
        xtmp.iloc[i][
            [
                "pred.causes_og",
                "pred.effects_og",
                "gold.causes",
                "gold.effects",
                "context",
                "causes_og_cont_ss",
                "effects_og_cont_ss",
            ]
        ]
    )

0


,4
pred.causes_og,recent generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat covid-19
pred.effects_og,we are keeping our fair value estimate for danaher at $160 per share after raising it in january 2021
gold.causes,recently generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat covid-19
gold.effects,we are keeping our fair value estimate for danaher at $160 per share after raising it in january 2021
context,We are keeping our fair value estimate for Danaher at $160 per share after raising it in January 2021 to reflect recently generated cash flows and our higher near-term outlook primarily for its tools used to diagnose and develop products to treat COVID-19.
causes_og_cont_ss,False
effects_og_cont_ss,True


1


,47
pred.causes_og,the rise of e-commerce and the coronavirus outbreak
pred.effects_og,an increased focus on supply chains has eroded this prior source of hesitation
gold.causes,the rise of e-commerce and the coronavirus outbreak
gold.effects,an increased focus on supply chains
context,"Whereas the relative complexity involved in developing a facility had served as a deterrent, an increased focus on supply chains due to the rise of e-commerce and the coronavirus outbreak has eroded this prior source of hesitation."
causes_og_cont_ss,True
effects_og_cont_ss,False


2


...

,1438
pred.causes_og,"on a consolidated basis, santander's diverse lending book, with exposure to markets that are not necessarily correlated, provides it with downside protection during downturns in the credit cycle of a region or asset class"
pred.effects_og,santander's earnings have shown the least volatility out of all the large european universal banks over the past two decades
gold.causes,"on a consolidated basis, santander's diverse lending book, with exposure to markets that are not necessarily correlated, provides it with downside protection during downturns in the credit cycle of a region or asset class"
gold.effects,santander's earnings have showed the least volatility out of all the large european universal banks over the past two decades
context,"However, on a consolidated basis, Santander's diverse lending book, with exposure to markets that are not necessarily correlated, provides it with downside protection during downturns in the credit cycle of a region or asset class. Consequently, Santander's earnings have showed the least volatility out of all the large European universal banks over the past two decades."
causes_og_cont_ss,True
effects_og_cont_ss,False


8


,1613
pred.causes_og,eda tools
pred.effects_og,chips that companies like cadence provide critical to customers
gold.causes,chip design would be virtually impossible without eda tools
gold.effects,tools that companies like cadence provide critical to customers
context,"Overall, chip design would be virtually impossible without EDA tools--this makes tools that companies like Cadence provide critical to customers."
causes_og_cont_ss,True
effects_og_cont_ss,False


9


,1870
pred.causes_og,the coronavirus restrictions
pred.effects_og,"the main business areas affected have been roaming, the business-to-business segment in the nordic markets and prepaid asian markets"
gold.causes,the coronavirus restrictions
gold.effects,"roaming, the business-to-business segment in the nordic markets and prepaid asian markets"
context,"The main business areas affected by the coronavirus restrictions have been roaming, the business-to-business segment in the Nordic markets and prepaid Asian markets."
causes_og_cont_ss,True
effects_og_cont_ss,False
